# Start with loading the glove embedding

In [1]:
import numpy as np 
%matplotlib inline

import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

import torch
from torch import nn
import math
from functools import partial
from pathlib import Path
from tqdm import tqdm
#import rich
from typing import List, Tuple, Optional, Dict, Any
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
#import transformers
#import tokenizers
#import datasets
#import zipfile
#from huggingface_hub import hf_hub_download
device = 'cpu'

In [2]:
target_file = "glove.6B.50d.txt"
vocabulary = []
vectors = []
emb_dic = {}
with open(target_file, "r", encoding="utf8") as f:
    for l in tqdm(f.readlines() ):
        word, *vector = l.split()
        vocabulary.append(word)
        vector_t = torch.tensor([float(v) for v in vector])
        vectors.append(vector_t)
        emb_dic[word] = vector_t
vectors = torch.stack(vectors)

100%|██████████| 400001/400001 [00:08<00:00, 44765.08it/s]


In [3]:
# somtehing from week 4
dim_emb = 50
class NewsEncoder(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.num_classes = num_classes
        self.activation_fn = nn.ReLU()
        self.mul_head_att = nn.MultiheadAttention(dim_emb,1)
        self.lin_q = nn.Linear(in_features=dim_emb,out_features=dim_emb)
        self.lin_k = nn.Linear(in_features=dim_emb,out_features=dim_emb)
        self.lin_v = nn.Linear(in_features=dim_emb,out_features=dim_emb)
        
        self.ff1 = nn.Linear(in_features=dim_emb, out_features=128)
        self.ff2 = nn.Linear(in_features=128, out_features=num_classes)

        #self.l_1 = nn.Linear(in_features=int(features_cat_size),
        #                  out_features=l_1_hidden_size)
        

    def forward(self, x): # x is a string of words - title
        words = x.split(" ")
        vectors = []
        for word in words:
            print(word)
            vectors.append(torch.Tensor(emb_dic[word.lower()]))
        #features_final = self.l_1(x)
        e_s = torch.stack(vectors)
        q = self.lin_q(e_s)
        k = self.lin_k(e_s)
        v = self.lin_v(e_s)
        attn_out, attn_weight = self.mul_head_att(q,k,v)
        #print(attn_out)
        cls_tok_emb = attn_out[0,:]
        print(cls_tok_emb)
        
        output = self.ff1(cls_tok_emb)
        output = self.activation_fn(output)
        output = self.ff2(output)
        
        return output
    


In [4]:
model = NewsEncoder(5) # TODO - I need to change this with a number that will correspond to 

In [5]:
model

NewsEncoder(
  (activation_fn): ReLU()
  (mul_head_att): MultiheadAttention(
    (out_proj): NonDynamicallyQuantizableLinear(in_features=50, out_features=50, bias=True)
  )
  (lin_q): Linear(in_features=50, out_features=50, bias=True)
  (lin_k): Linear(in_features=50, out_features=50, bias=True)
  (lin_v): Linear(in_features=50, out_features=50, bias=True)
  (ff1): Linear(in_features=50, out_features=128, bias=True)
  (ff2): Linear(in_features=128, out_features=5, bias=True)
)

In [6]:
test1 = "Breaking news about this"
test2 = "This has happened again"
#test3 = "Natascha var ikke den foerste"
#test4 = "Kun Star Wars tjente mere"

In [7]:
attn_output = model.forward(test2)
shape = attn_output.shape
attn_output.reshape(shape)

This
has
happened
again
tensor([ 0.0623, -0.1269,  0.1517,  0.0182, -0.0764, -0.1888,  0.0502,  0.1932,
        -0.0752, -0.2148,  0.0839, -0.0535,  0.0456, -0.0375,  0.2603, -0.1023,
        -0.4326, -0.0162, -0.2151,  0.2617, -0.4282,  0.0268,  0.0388, -0.2200,
        -0.0063,  0.1313,  0.3021, -0.2832,  0.1869,  0.2080, -0.0155, -0.3546,
         0.0543,  0.0897, -0.0034,  0.3228, -0.1696, -0.0180,  0.0544,  0.0395,
        -0.2224,  0.0550, -0.1932,  0.0877,  0.2250, -0.0268, -0.0923, -0.1682,
        -0.1431, -0.4634], grad_fn=<SliceBackward0>)


tensor([-0.0823,  0.0722,  0.0352,  0.0860, -0.0197], grad_fn=<ViewBackward0>)

# brand new encoder

In [8]:
class MyEmbeddingLayer(nn.Module):
    def __init__(self,emb_dim, emb_dic):
        super().__init__()
        self.emb_dim = emb_dim
        self.emb_dic = emb_dic
        
    def forward(self,text):
        input_shape = text.shape
        titles = text.flatten()
        output = []
        for title in titles:
            words = title.split(" ")
            for word in words:
                output.append(torch.Tensor(self.emb_dic[word.lower()]))
            
        output = torch.stack(output)
        output = output.reshape(input_shape + (-1,self.emb_dim))
        return output
            

In [9]:
test = MyEmbeddingLayer(50, emb_dic)
input = np.array([test1,test2])
print(input.shape)
#input = torch.from_numpy(input)
test.forward(input)

(2,)


tensor([[[-7.5307e-01, -1.4580e-02, -7.5133e-02,  1.0766e-01, -1.1661e-02,
           4.1505e-01,  7.2856e-02,  4.9378e-01,  1.4302e-01,  2.7014e-01,
          -2.9928e-01, -4.9729e-01, -1.1358e+00,  5.7973e-01,  5.5898e-01,
          -3.1587e-01, -5.7323e-02, -7.2313e-01, -4.0444e-01, -2.3284e-01,
           1.8744e-01,  3.2356e-01, -1.6800e-02, -3.3481e-01,  2.2256e-01,
          -1.3342e+00,  4.9411e-01,  8.8150e-02,  8.0475e-01, -1.7204e-01,
           2.7073e+00,  1.2706e-01,  9.3040e-02,  3.3197e-01, -7.0747e-01,
           2.3589e-01,  3.0587e-01,  1.6879e-01, -1.8252e-01, -1.2134e-01,
          -2.3045e-01, -1.5842e-01,  5.3029e-02,  4.9780e-01,  1.1690e-01,
           4.4732e-02,  2.3904e-01, -4.5940e-01, -4.5749e-01, -6.2330e-01],
         [-2.0825e-01,  4.7786e-01,  5.2196e-01,  1.0587e+00, -1.0045e-01,
          -1.1269e+00, -1.2581e+00, -1.1041e-01, -7.4125e-02, -7.7976e-01,
          -3.7942e-01, -2.4860e-01, -3.9224e-01,  4.2972e-01,  9.8060e-01,
           1.2668e-01, -

### version 2

In [10]:
# somtehing from week 4
dim_emb = 50
class MyNewsEncoder(nn.Module):
    def __init__(self, out_dimension, head_count=1):
        super().__init__()
        self.out_dimension = out_dimension
        self.activation_fn = nn.Tanh()
        self.embedding = MyEmbeddingLayer(dim_emb,emb_dic)
        self.softmax_dim1 = nn.Softmax(dim=1)
        self.softmax = nn.Softmax()
        self.v_net = nn.Linear(dim_emb, dim_emb)
        #--------
        self.mul_head_att = nn.MultiheadAttention(dim_emb,1)
        self.lin_q = nn.Linear(in_features=dim_emb,out_features=dim_emb)
        self.lin_k = nn.Linear(in_features=dim_emb,out_features=dim_emb)
        self.lin_v = nn.Linear(in_features=dim_emb,out_features=dim_emb)
        
        self.ff1 = nn.Linear(in_features=dim_emb, out_features=128)
        self.ff2 = nn.Linear(in_features=128, out_features=out_dimension)

    def forward(self, x): # x is a string of words - title
        #words = x.split(" ")
        #vectors = []
        #for word in words:
        #    print(word)
        #    vectors.append(torch.Tensor(emb_dic[word.lower()]))
        #features_final = self.l_1(x)
        e_s = self.embedding(x)
        q = self.lin_q(e_s)
        k = self.lin_k(e_s)
        v = self.lin_v(e_s)
        a_k = self.get_A_Matrix(e_s, q)
        h_ks = self.get_h_head_vectors(v,a_k,e_s)
        h = self.combine_heads(h_ks)
        
        a_w = q.T @ self.activation_fn(self.v_net(h))
        a_w = self.softmax(a_w)
        
        
        attn_out, attn_weight = self.mul_head_att(q,k,v)
        #print(attn_out)
        cls_tok_emb = attn_out[0,:]
        print(cls_tok_emb)
        
        output = self.ff1(cls_tok_emb)
        output = self.activation_fn(output)
        output = self.ff2(output)
        
        return output
    
    def get_A_Matrix(e,Q):
        matr_mult = e.T @ Q.T @ e
        a = softmax_dim1(matr_mult)
        return a
    
    def get_h_head_vectors(V, a, e):
        inner_dot = a @ e # TODO check if the dimensions really make dot product
        h = V @ inner
        return h
    
    def combine_heads(h_ks):
        return h_ks # TODO change this when I will have multi head

        
    


### version 3

In [11]:
# somtehing from week 4
dim_emb = 50
class MyNewsEncoder(nn.Module):
    def __init__(self, out_dimension, head_count=1):
        super().__init__()
        self.out_dimension = out_dimension
        self.activation_fn = nn.Tanh()
        self.embedding = MyEmbeddingLayer(dim_emb,emb_dic)
        self.softmax_dim1 = nn.Softmax(dim=1) # TODO do I go for the correct dimension?
        self.softmax = nn.Softmax(dim=1)
        #--------
        self.lin_qk = nn.Linear(dim_emb, dim_emb, bias=False)
        self.lin_q = nn.Linear(in_features=dim_emb,out_features=1)
        self.lin_vk = nn.Linear(in_features=dim_emb,out_features=dim_emb, bias=False)
        self.lin_vw = nn.Linear(in_features=dim_emb,out_features=dim_emb)

    def forward(self, x): # x is a string of words - title
        e_s = self.embedding(x)
        
        # starting single head
        qe = self.lin_qk(e_s)
        et_qt = qe @ e_s.transpose(-2,-1)
        ak = self.softmax_dim1(et_qt)
        print((ak @ e_s).shape)
        hk = self.lin_vk(ak @ e_s)
        h = hk # TODO make it multi head
        print('h shape', h.shape)
        # end of single head
        
        aw = self.lin_q(self.activation_fn(self.lin_vw(h)))
        print('AW',aw.shape)
        aw = self.softmax(aw)
        print(aw.shape, h.shape)
        r = aw.transpose(-2,-1) @ h # just confirmed by calculation that this seems correct
        return r.reshape(-1,50)
    


### version 4

In [12]:
dim_emb = 50

class SelfAttHead(nn.Module):
    def __init__(self, dim_emb, head_out):
        super().__init__()
        self.lin_qk = nn.Linear(dim_emb, dim_emb, bias=False)
        self.softmax_dim1 = nn.Softmax(dim=1) # TODO do I go for the correct dimension?
        self.lin_vk = nn.Linear(in_features=dim_emb,out_features=head_out, bias=False)
        
    def forward(self,x):
        qe = self.lin_qk(x)
        et_qt = qe @ x.transpose(-2,-1)
        ak = self.softmax_dim1(et_qt)
        hk = self.lin_vk(ak @ x)
        return hk
    
    
class MyNewsEncoder(nn.Module):
    def __init__(self, embedding_dimension, head_count=1):
        super().__init__()
        assert embedding_dimension % head_count == 0, "embeding must be divisible by heads"
        self.head_out = embedding_dimension // head_count
        self.activation_fn = nn.Tanh()
        self.embedding_layer = nn.Embeding(200,dim_emb)
        self.embedding = MyEmbeddingLayer(dim_emb,emb_dic)
        self.softmax = nn.Softmax(dim=1)
        #--------
        self.lin_vw = nn.Linear(in_features=dim_emb,out_features=dim_emb)
        self.lin_q = nn.Linear(in_features=dim_emb,out_features=1, bias=False)
        
        self.selfAtt = nn.ModuleList([SelfAttHead(dim_emb, self.head_out) for i in range(head_count)])

    def forward(self, x): # x is a string of words - title
        e_s = self.embedding(x)
        
        hk = []
        for head in self.selfAtt:
            att = head(e_s)
            hk.append(att)
        h = torch.cat(hk, 2)
        print(h.shape)
        
        aw = self.lin_q(self.activation_fn(self.lin_vw(h)))
        aw = self.softmax(aw)
        r = aw.transpose(-2,-1) @ h # just confirmed by calculation that this seems correct
        return r.reshape(-1,50)
    


In [13]:
model = MyNewsEncoder(dim_emb, head_count=10)
model(input)

torch.Size([2, 4, 50])


tensor([[ 0.1049, -0.4537, -0.1356, -0.1396,  0.0383, -0.3921,  0.3203, -0.4113,
          0.2908, -0.4372,  0.3781,  0.6186,  0.2963,  0.4645,  0.1341,  0.1073,
          0.1490,  0.2557,  0.0189,  0.0372,  0.0575,  0.0744,  0.1842,  0.4117,
          0.3078,  0.1159,  0.3378,  0.2334,  0.1820, -0.0393, -0.3553, -0.0286,
          0.1830, -0.0683,  0.7422, -0.4820, -0.2098,  0.1134,  0.2645,  0.6888,
          0.3641,  0.0550, -0.0983,  0.2359, -0.3176,  0.3771,  0.4073,  0.1636,
          0.0647,  0.2981],
        [-0.0721, -0.3664, -0.2688, -0.1042, -0.0194, -0.2130,  0.2629, -0.1272,
          0.0166, -0.1312,  0.3772,  0.5362,  0.0383,  0.3289,  0.1689,  0.0200,
          0.1522,  0.2349, -0.0689, -0.2110,  0.2611, -0.1230,  0.0137,  0.5855,
          0.3744,  0.2682,  0.3069,  0.2010,  0.0873, -0.1775, -0.3700, -0.0265,
          0.0632,  0.1766,  0.7144, -0.3285, -0.1967,  0.0916,  0.2665,  0.6822,
          0.6188,  0.0891,  0.1862,  0.1448, -0.3254,  0.3256,  0.1844,  0.2128,


In [14]:
simplemat = torch.rand(2,3)
simplemat, simplemat.sum(dim=1)

(tensor([[0.0840, 0.0074, 0.7121],
         [0.4423, 0.5169, 0.7201]]),
 tensor([0.8036, 1.6793]))

## translating

In [21]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

import torch
from torch import nn
import math
from functools import partial
from pathlib import Path
from tqdm import tqdm
import rich
from typing import List, Tuple, Optional, Dict, Any
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import transformers
import tokenizers
import datasets
import zipfile
from huggingface_hub import hf_hub_download
import pyarrow.parquet as pq


In [22]:
task_name = f"translation_da_to_en"
model_name = f"Helsinki-NLP/opus-mt-da-en"
translator  = transformers.pipeline(task_name, model=model_name)

/run/media/lukisko/Windows-SSD/DTU/SEMESTER 2/deep-learning/project/env/lib/python3.12/site-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [23]:
translator(['Natascha var ikke den første','Kun Star Wars tjente mere'])

[{'translation_text': "Natascha wasn't the first."},
 {'translation_text': 'Only Star Wars made more'}]

In [24]:
table_articles = pq.read_table('ebnerd_small/articles.parquet')
df_articles = table_articles.to_pandas()

In [29]:
emb_dic['t']

tensor([-0.3792,  0.6185,  0.9593,  0.9040,  0.3681,  0.0230,  0.1680, -1.5309,
        -0.0605, -0.2500,  0.1503,  0.3197, -0.6891, -0.7863, -0.0158,  0.5053,
        -0.8473, -0.1235,  0.0785, -0.9602, -0.5431, -0.3352,  0.3893,  0.1946,
        -1.1688, -0.8661, -0.3918,  0.2418,  0.3286, -0.7875,  2.4884,  0.7102,
        -0.5311,  0.8959, -0.2305, -0.8202,  0.3442, -0.9687, -0.1514, -0.4491,
         0.8951, -0.0166, -0.2749,  0.2795,  0.7794, -0.3194,  0.1676, -0.6252,
         0.0533,  0.6202])

In [30]:
df_articles.shape

(20738, 21)

# links
https://sanjayasubedi.com.np/deeplearning/multihead-attention-from-scratch/

In [43]:
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=5e-4)#, weight_decay=0.3)

NameError: name 'optim' is not defined

In [ ]:
# Test the forward pass with dummy data
out = model([])
print("Output shape:", out.size())
print(f"Output logits:\n{out.detach().cpu().numpy()}")
print(f"Output probabilities:\n{out.softmax(1).detach().cpu().numpy()}")